In [1]:
import torch
from prototypes.deeplearning.dataloader.IsicDataLoader import LoadDataVectors
from prototypes.deeplearning.models import Resnet50Prototype1, Resnet50Prototype2
import os
from prototypes.utility.data import ProjectConfiguration

In [3]:
model_loaded = Resnet50Prototype1(n_classes=1)
model_loaded.load_state_dict(torch.load(os.path.join("../checkpoint_resnet50_mix_up", "0.1.0_best.pt"), weights_only=True))

<All keys matched successfully>

# Predict

In [4]:
config = ProjectConfiguration("../config.json")

In [5]:
dataloader = LoadDataVectors(hd5_file_path=os.path.join(config.get_value("DATASET_PATH"), "train-image.hdf5"),
                             transform=model_loaded.weights.transforms(),
                             metadata_csv_path=config.get_value("TRAIN_METADATA"))

# sampler = torch.utils.data.RandomSampler(dataloader, num_samples=config.get_value("VAL_SAMPLE_SIZE"))
# dataloader = torch.utils.data.DataLoader(dataloader, batch_size=128, shuffle=False, num_workers=16, sampler=sampler)
dataloader = torch.utils.data.DataLoader(dataloader, batch_size=128, shuffle=False, num_workers=16)

In [9]:
from tqdm.auto import tqdm

model_loaded = model_loaded.cuda()

y_pred = []
y_true = []

for batch in tqdm(dataloader):
    x = batch[0].cuda()
    y = batch[1].cuda()
    
    with torch.no_grad():
        y_pred.extend(model_loaded(x).cpu().numpy())
        y_true.extend(y.cpu().numpy())

  0%|          | 0/3134 [00:00<?, ?it/s]

In [10]:
import pandas as pd

solution_df = pd.DataFrame(zip(y_true, [e[0] for e in y_true]), columns=['isic_id', 'target'])
submission_df = pd.DataFrame(zip(y_pred, [e[0] for e in y_pred]), columns=['isic_id', 'target'])

In [11]:
from prototypes.deeplearning.trainner import score

In [12]:
submission_df.head()

,isic_id,target
0,[0.53557104],0.535571
1,[0.52318925],0.523189
2,[0.5037488],0.503749
3,[0.51554567],0.515546
4,[0.52910924],0.529109


In [13]:
score(solution=solution_df, submission=submission_df, row_id_column_name="isic_id", min_tpr=0.80)

0.024961658304571734